In [ ]:
from google.cloud import bigquery
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from pandas.plotting import bootstrap_plot
import seaborn as sns
from scipy import stats
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

In [ ]:
#Initialize the BQ Client
client = bigquery.Client()

#SQL Query for the data you are pulling out of BigQuery
sql = ("SELECT Brand, Department, Engineered_Review_Text AS Text FROM `your_gcp_project.your_dataset.your_bazaarvoice_table`")

#Execute the query and load it into a DataFrame
df = client.query(sql).to_dataframe()

In [ ]:
#Examine records
df.head()

In [ ]:
#Initialize the vader
vader = SentimentIntensityAnalyzer()

#callable function to process each Text
def sentiment_analyzer_scores(text):
    score = vader.polarity_scores(text) 
    print("{:-<40} {}".format(text, str(score)))

In [ ]:
#counter
i=0
#empty list
vadervalues = [ ]
#call the function for each row in the dataframe and return only the compound score
while (i<len(df)):
    k = vader.polarity_scores(df.iloc[i]['Text'])
    vadervalues.append(k['compound'])
    i = i+1

#add the compound vader score as a column on the original frame
vadervalues = np.array(vadervalues)
df['vader'] = vadervalues

In [ ]:
#examine records
df.head()

In [ ]:
#check average vader score by merchandise department category
df_department_mean = pd.DataFrame(df.groupby(['Department'])['vader'].mean())

In [ ]:
#show ordered results from most positive to least positive
df_top_departments = df_department_mean.sort_values(['vader'], ascending=False)
df_top_departments.head(100)

In [ ]:
#plot the sentiment by merchandise department
df.boxplot(by='Department')

In [ ]:
#ordered list of merchandise departments by positive vader score descending
my_order = df.groupby(by=['Department'])['vader'].median().sort_values(ascending=False).index
#seaborn produces a cleaner plot
sns.set(rc={'figure.figsize':(25,10)})
brandbox = sns.boxplot(x="Department", y="vader", fliersize=1 , data=df, palette="BuGn_r", showfliers=False, order=my_order)
brandbox.set_xticklabels(brandbox.get_xticklabels(), rotation=45, ha="right")

In [ ]:
#get average vader score by Brand Name
df_brand_mean = pd.DataFrame(df.groupby(['Brand'])['vader'].mean())

In [ ]:
#ordered list of TOP 10 brand names with BEST sentiment score
df_top_brands = df_brand_mean.sort_values(['vader'], ascending=False)
df_top_brands.head(10)

In [ ]:
#ordered list of BOTTOM 10 brand names with WORST sentiment score
df_bot_brands = df_brand_mean.sort_values(['vader'], ascending=True)
df_bot_brands.head(10)

In [ ]:
#Histogram showing the overall company sentiment scores
df_brand_mean.plot.hist(stacked=True, bins=50)

In [ ]:
#Histogram showing overall company with overlayed kde density line
sns.distplot(df_brand_mean)

In [ ]:
#Density PLot showing overall company sentiment scores
df_brand_mean.plot.kde()

In [ ]:
#bootstrap vis of overall scores
bootstrap_plot(df_brand_mean['vader'], size=100, samples=50)

In [ ]:
#Example of word cloud to see what customers are saying overall
wordcloud = WordCloud(
    width = 1000,
    height = 500,
    background_color = 'black',
    stopwords = STOPWORDS).generate(str(df['Text']))

fig = plt.figure(
    figsize = (40, 30),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
#Example of word cloud showing only one Merchandise Department we want to examine
DepartmentX = df.loc[df['Department'] == "DepartmentX"]

DepartmentX_wordcloud = WordCloud(
    width = 3000,
    height = 2000,
    background_color = 'black',
    stopwords = STOPWORDS).generate(str(DepartmentX['Text']))

fig = plt.figure(
    figsize = (40, 30),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(DepartmentX_wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
#Example of word cloud showing only one Merchandise Department we want to examine
DepartmentY = df.loc[df['Department'] == "DepartmentY"]

DepartmentY_wordcloud = WordCloud(
    width = 3000,
    height = 2000,
    background_color = 'black',
    stopwords = STOPWORDS).generate(str(DepartmentY['Text']))

fig = plt.figure(
    figsize = (40, 30),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(DepartmentY_wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
#Examine the highest rated brand by name
Highest_Rated_Brand_df = df.loc[df['Brand'] == "Highest Rated Brand Name"]

In [ ]:
#hist
Highest_Rated_Brand_df.plot.hist(stacked=True, bins=50)

In [ ]:
#density
sns.distplot(Highest_Rated_Brand_df['vader'], kde=True);

In [ ]:
#Examine an interesting brand name by name
Another_Brand_df = df.loc[df['Brand'] == "Another Brand Name"]

In [ ]:
sns.distplot(Another_Brand_df['vader'], kde=True);

In [ ]:
#Examine an interesting brand name word cloud
ANOTHER_BRAND_wordcloud = WordCloud(
    width = 3000,
    height = 2000,
    background_color = 'black',
    stopwords = STOPWORDS).generate(str(Another_Brand_df['Text']))

fig = plt.figure(
    figsize = (40, 30),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(ANOTHER_BRAND_wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
#Drill into a specific merchandise department name only and split by another attribute
#Derive Attribute X by similarly named Departments
ATTRIBUTEX_DepartmentName = df.loc[df['Department'] == "ATTRIBUTEX Department Name"]
ATTRIBUTEX_DepartmentName['ATTRIBUTE']= 'ATTRIBUTEX'
ATTRIBUTEX_DepartmentName['Dept']= 'Department Name'
df_XA_mean = pd.DataFrame(ATTRIBUTEX_DepartmentName.groupby(['ATTRIBUTE','Brand'])['vader'].mean())
df_top_XA_brands = df_XA_mean.sort_values(['vader'], ascending=False)
df_top_XA_brands.head(100)

In [ ]:
#COLLECT THE OTHER ATTRIBUTE
#Derive Attribute Y by similarly named Departments
ATTRIBUTEY_DepartmentName = df.loc[df['Department'] == "ATTRIBUTEY Department Name"]
ATTRIBUTEY_DepartmentName['ATTRIBUTE']= 'ATTRIBUTEY'
ATTRIBUTEY_DepartmentName['Dept']= 'Department Name'
df_YA_mean = pd.DataFrame(ATTRIBUTEX_DepartmentName.groupby(['ATTRIBUTE','Brand'])['vader'].mean())
df_top_YA_brands = df_YA_mean.sort_values(['vader'], ascending=False)
df_top_YA_brands.head(100)

In [ ]:
#UNION ATTRIBUTE X AND Y df together
df_ATTRIBUTE = pd.concat([ATTRIBUTEY_DepartmentName, ATTRIBUTEX_DepartmentName])
df_ATTRIBUTE.head()

In [ ]:
#Plot Sentiment Scores By Brand Comparing Attribute X and Y for each brand
sns.boxplot(x="Brand", y="vader",
            hue="ATTRIBUTE", palette=["m", "g"],
            data=df_ATTRIBUTE)
sns.despine(offset=10, trim=True)

In [ ]:
#Cleaner SNS version
df_ATTRIBUTE_order = df_ATTRIBUTE.groupby(by=['Brand'])['vader'].median().sort_values(ascending=False).index

sns.set(rc={'figure.figsize':(25,10)})
brandbox = sns.boxplot(x="Brand", y="vader", 
                       data=df_ATTRIBUTE, 
                       showfliers=False, 
                       order=df_ATTRIBUTE_order, 
                       hue="ATTRIBUTE", 
                       palette=["m", "g"])
brandbox.set_xticklabels(brandbox.get_xticklabels(), rotation=45, ha="right")